Thermal history analysis
========================

This example shows how you can use PyAdditive to determine thermal
history during a build using a simulated coaxial average sensor.

Units are SI (m, kg, s, K) unless otherwise noted.


Perform required import and connect
===================================

Perform the required import and connect to the Additive service.


In [ ]:
from ansys.additive.core import (
    Additive,
    AdditiveMachine,
    BuildFile,
    CoaxialAverageSensorInputs,
    MachineType,
    Range,
    SimulationError,
    StlFile,
    ThermalHistoryInput,
)

additive = Additive()

Select model
============

Select the geometry model. Currently, PyAdditive supports two types of
geometry specifications,
`STLFile <ansys.additive.core.geometry_file.StlFile>`{.interpreted-text
role="class"} and
`BuildFile <ansys.additive.core.geometry_file.BuildFile>`{.interpreted-text
role="class"}.

You can download example build and STL files by importing the `examples`
module.


In [ ]:
import ansys.additive.core.examples as examples

# Create an ``StlFile`` object.
stl_name = examples.download_10mm_cube()
stl_file = StlFile(stl_name)

# Or, create a ``BuildFile`` object.
build_file_name = examples.download_small_wedge_slm_build_file()
build_file = BuildFile(MachineType.SLM, build_file_name)

Select material
===============

Select a material. You can use the
`materials_list() <ansys.additive.additive.Additive.materials_list>`{.interpreted-text
role="meth"} method to obtain a list of available materials.


In [ ]:
additive.materials_list()

You can obtain the parameters for a single material by passing a name
from the materials list to the
`material() <ansys.additive.additive.Additive.material>`{.interpreted-text
role="meth"} method.


In [ ]:
material = additive.material("17-4PH")

Specify machine parameters
==========================

Specify machine parameters by first creating an
`AdditiveMachine <ansys.additive.core.machine.AdditiveMachine>`{.interpreted-text
role="class"} object then assigning the desired values. All values are
in SI units (m, kg, s, K) unless otherwise noted.


In [ ]:
machine = AdditiveMachine()

# Show available parameters
print(machine)

Set laser power and scan speed
==============================

Set the laser power and scan speed.


In [ ]:
machine.scan_speed = 1  # m/s
machine.laser_power = 500  # W

Specify inputs for thermal history simulation
=============================================

Thermal history is simulated for the given geometry over a range of
heights in the Z dimension. More than one range can be specified. Each
range is specified with a
`Range <ansys.additive.core.thermal_history.Range>`{.interpreted-text
role="class"} object. The ranges are assigned to a
`CoaxialAverageSensorInputs <ansys.additive.core.thermal_history.CoaxialAverageSensorInputs>`{.interpreted-text
role="class"} object which also includes a sensor radius. The
`CoaxialAverageSensorInputs` object is assigned to a
`ThermalHistoryInput <ansys.additive.core.thermal_history.ThermalHistoryInput>`{.interpreted-text
role="class"} object.


In [ ]:
# Values are in meters
sensor_inputs = CoaxialAverageSensorInputs(
    radius=5e-4,
    z_heights=[Range(min=1e-3, max=1.1e-3), Range(min=6.5e-3, max=6.6e-3)],
)

input = ThermalHistoryInput(
    machine=machine,
    material=material,
    id="thermal-history-example",
    geometry=stl_file,
    coax_ave_sensor_inputs=sensor_inputs,
)

Run simulation
==============

Use the
`simulate() <ansys.additive.core.additive.Additive.simulate>`{.interpreted-text
role="meth"} method of the `additive` object to run the simulation. The
returned object is a either a
`ThermalHistorySummary <ansys.additive.core.thermal_history.ThemalHistorySummary>`{.interpreted-text
role="class"} object or a
`SimulationError <ansys.additive.core.simulation.SimulationError>`{.interpreted-text
role="class"}.


In [ ]:
summary = additive.simulate(input)
if isinstance(summary, SimulationError):
    raise Exception(summary.message)

Plot thermal history
====================

Plot the thermal history using PyVista.


In [ ]:
import glob
import os

import pyvista as pv

vtk_files = glob.glob(os.path.join(summary.coax_ave_output_folder, "*.vtk"))
for file in vtk_files:
    plotter = pv.Plotter(window_size=[512, 512])
    plotter.add_mesh(pv.read(file))
    title = os.path.splitext(os.path.basename(file))[0]
    plotter.add_title(title, font_size=8)
    plotter.show()